# Dory's Story

## Import Packages

In [9]:
import pandas as pd
import zipfile
import requests
import io
import arcpy

## Set Working Folder

In [13]:
dory_working = r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory"

## Call in XY Coordinates

In [3]:
startXY = pd.read_csv(r"C:\Users\tande\Documents\GitHub\GIS5571\Lab 02\Dory Farm XY.csv")
endXY = pd.read_csv(r"C:\Users\tande\Documents\GitHub\GIS5571\Lab 02\North Picnic XY.csv")

## Import Data 

In [17]:
# County boundaries
county_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/shp_bdry_counties.zip'
county_post_request = requests.post(county_url)
county_post_request

county_zipped = zipfile.ZipFile(
        io.BytesIO(
            county_post_request.content)
)

county_zipped.extractall(dory_working)

# MN Landcover
landcover_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
landcover_post_request = requests.post(landcover_url)
landcover_post_request

landcover_zipped = zipfile.ZipFile(
        io.BytesIO(
            landcover_post_request.content)
)

landcover_zipped.extractall(dory_working)

# Water routes

water_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'
water_post_request = requests.post(water_url)
water_post_request

water_zipped = zipfile.ZipFile(
        io.BytesIO(
            water_post_request.content)
)

water_zipped.extractall(dory_working)

# Elevation 

elevation_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
elevation_post_request = requests.post(elevation_url)
elevation_post_request

elevation_zipped = zipfile.ZipFile(
        io.BytesIO(
            elevation_post_request.content)
)

elevation_zipped.extractall(dory_working)

roads_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip'
roads_post_request = requests.post(roads_url)
roads_post_request

roads_zipped = zipfile.ZipFile(
        io.BytesIO(
            roads_post_request.content)
)

roads_zipped.extractall(dory_working)

In [ ]:
# XY Coordinates to Points

arcpy.management.XYTableToPoint("Dory Farm XY.csv", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DoryFarmXY_XYTableToPoint", "X", "Y", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
arcpy.management.XYTableToPoint("North Picnic XY.csv", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\NorthPicnicXY_XYTableToPoint", "X", "Y", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

## Clipping to Extent

In [15]:
# The extent I am using are the 3 counties surrounding the points: Olmstead, Winona, and Wabasha

# Selecting this extent
arcpy.management.SelectLayerByAttribute("County_Boundaries_in_Minnesota", "NEW_SELECTION", "COUNTY_NAM = 'Wabasha' And COUNTY_NAM = 'Olmsted' And COUNTY_NAM = 'Winona'", None)

# Clipping the boundaries to the selected boundaries
arcpy.analysis.PairwiseClip("County_Boundaries_in_Minnesota", "County_Boundaries_in_Minnesota", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Selected_County_Boundaries", None)

# Clipping streams to selected boundaries
arcpy.analysis.PairwiseClip("streams_with_strahler_stream_order", "Selected_County_Boundaries", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_st_PairwiseClip", None)

# Clipping DEM
arcpy.management.Clip("digital_elevation_model_30m", "524956.361 4853464.52 637943.77 4922616.7", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\digital_elevation_model_Clip", "Selected_County_Boundaries", "32767", "NONE", "NO_MAINTAIN_EXTENT")

# Clipping Land Cover
arcpy.management.Clip("NLCD 2019 - Land Cover", "524956.361 4853464.52 637943.77 4922616.7", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\LandcoverRaster_Clip", "Selected_County_Boundaries", "255", "NONE", "NO_MAINTAIN_EXTENT")

# Clipping roads
arcpy.analysis.PairwiseClip("STREETS_LOAD", "Selected_County_Boundaries", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_PairwiseClip", None)

<Result 'C:\\Users\\tande\\Documents\\GIS5571\\GIS5571 Lab 2\\Dory\\GIS5571_Lab2Part2\\GIS5571_Lab2Part2.gdb\\Selected_County_Boundaries'>

## Preparing Each Clipped Dataset for Raster Calculator

In [ ]:
# Reclassifying Ref https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/creating-a-cost-surface-raster.htm

# For commonalities in rasters - using 9 classifications and Red-Yellow-Green continuous symbology 

# I don't think symbology matters but just doing this for good measure

### County Boundaries

In [ ]:
# Only used for clipping the extent

### Road Network

In [ ]:
# Create a 100m buffer around all roads in the extent

arcpy.analysis.Buffer("STREETS_LOAD_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_Pairwise_Buffer", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")arcpy.analysis.Buffer("STREETS_LOAD_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_Pairwise_Buffer", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

# Convert to raster

arcpy.conversion.FeatureToRaster("STREETS_LOAD_Pairwise_Buffer", "STREET_NAM", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Feature_STRE1", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\digital_elevation_model_Clip")

# Reclassify to 9 classes

out_raster = arcpy.sa.Reclassify("Feature_STRE1", "Value", "1 452.333333 1;452.333333 903.666667 2;903.666667 1355 3;1355 1806.333333 4;1806.333333 2257.666667 5;2257.666667 2709 6;2709 3160.333333 7;3160.333333 3611.666667 8;3611.666667 4063 9", "NODATA"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2")

# Ready for Cost Surface

# Final raster name 'Reclass_Feat2'

### Streams

In [ ]:
# Create a 100m buffer around all streams in the extent

arcpy.analysis.PairwiseBuffer(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_st_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_s_PairwiseBuffe", "100 Meters", "NONE", None, "PLANAR", "0 Meters")

# Finding where rivers and roads intersect

arcpy.analysis.PairwiseIntersect("STREETS_LOAD_Pairwise_Buffer;streams_with_s_PairwiseBuffe", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_STREAMS_INTERSECT", "ALL", None, "INPUT")

# Erasing rivers where roads overlap

arcpy.analysis.PairwiseErase("STREETS_STREAMS_INTERSECT", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_s_PairwiseBuffe", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_STREAM_PairwiseErase", None)

# Converting to raster

arcpy.conversion.FeatureToRaster("streams_with_s_PairwiseBuffe", "SO_VALUE", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Stream_Raster", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\digital_elevation_model_Clip")

# Reclassify the raster

out_raster = arcpy.sa.Reclassify("Stream_Raster", "Value", "1 1;2 2;3 3;4 4;5 5;6 6;8 7", "NODATA"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Stre3")

# Raster calculator to eliminate null values

output_raster = arcpy.ia.RasterCalculator('Con(IsNull("Reclass_Stre3"),1, "Reclass_Stre3")'); output_raster.save(r"c:\users\tande\documents\gis5571\gis5571 lab 2\dory\gis5571_lab2part2\gis5571_lab2part2.gdb\t_coni_raste")

# Ready for Cost Surface

# Final raster name 't_coni_raste'

### Elevation

In [ ]:
# Calculate Slope

out_raster = arcpy.sa.Slope("digital_elevation_model_Clip", "DEGREE", 1, "PLANAR", "METER"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Slope_digita1")

# Reclassify

out_raster = arcpy.sa.Reclassify("Slope_digita1", "Value", "0 4.669600 1;4.669600 9.961813 2;9.961813 16.187945 3;16.187945 23.347998 4;23.347998 31.441971 5;31.441971 39.847250 6;39.847250 48.252530 7;48.252530 56.346502 8;56.346502 79.383194 9", "NODATA"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1")

# Ready for Cost Surface

# Final raster name 'Reclass_Slop1'

### Land Cover

In [ ]:
# Reclassify 

out_raster = arcpy.sa.Reclassify("LandcoverRaster_Clip", "Value", "11 1;21 2;22 3;23 4;24 5;31 6;41 7;42 8;43 9;52 10;71 11;81 12;82 13;90 14;95 15", "NODATA"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1")

# Ready for Cost Surface

# Final raster name 'Reclass_Land1'

## Cost Surface Analysis

In [ ]:
# Use Raster Calculator to Combine All Rasters

output_raster = arcpy.ia.RasterCalculator(' "Reclass_Feat2" +  "t_coni_raste" +  "Reclass_Slop1" +  "Reclass_Land1"'); output_raster.save(r"c:\Users\tande\documents\GIS5571\gis5571 lab 2\Dory\gis5571_lab2part2\gis5571_lab2part2.gdb\complete_raster")

# Weighted Overlay to try out different weights

## 25% weight for each raster

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\t_coni_raste' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighte_Recl2")

## 10% Roads, 10% Streams, 40% Elevation, 40% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\t_coni_raste' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighte_Recl2")

## 40% Roads, 40% Streams, 10% Elevation, 10% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\t_coni_raste' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighte_Recl2")

## %10 Roads, 40% Streams, 10% Elevation, 40% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\t_coni_raste' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighte_Recl2")

## 40% Roads, 10% Streams, 40% Elevation, 10% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Feat2' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\t_coni_raste' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Slop1' 40 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Reclass_Land1' 10 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 1; 11 1; 12 1; 13 1; 14 1; 15 1; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighte_Recl2")

# I heard from others that we are supposed to
#actually find the best path from pt A to pt B
#but I wasn't able to do this so I am just showing the 
# different weights